### Databaser - Labb 2

# Bokhandel - Applikation

In [12]:
from sqlalchemy import create_engine, MetaData, select, func
from sqlalchemy.engine import URL
from urllib.parse import unquote

server_name   = 'localhost'
database_name = 'labb2'

connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes"
url_string        = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')

try:    
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to {database_name}!')
except Exception as e:
    print('Error while connecting to database:\n')
    print(e)

Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER=localhost;DATABASE=labb2;Trusted_Connection=yes 

Successfully connected to labb2!


In [17]:
import pandas as pd

metadata = MetaData()

metadata.reflect(bind=engine)

books_table = metadata.tables['Böcker']
authors_table = metadata.tables['Författare']
stock_table = metadata.tables['LagerSaldo']
shops_table = metadata.tables['Butiker']

def search_books_by_title(title):
    with engine.connect() as connection:
        stmt = (
            select(
                books_table.c.ISBN13,
                books_table.c.Titel,
                books_table.c.Språk,
                books_table.c.Pris,
                books_table.c.Utgivningsdatum,
                authors_table.c.Förnamn,
                authors_table.c.Efternamn,
                func.concat(authors_table.c.Förnamn, ' ', authors_table.c.Efternamn).label('Författare'),
                stock_table.c.Antal.label('LagerAntal')
            )
            .select_from(books_table
                .join(authors_table, books_table.c.FörfattareId == authors_table.c.Id)
                .join(stock_table, books_table.c.ISBN13 == stock_table.c.ISBN13)
                .join(shops_table, stock_table.c.ButikId == shops_table.c.Id)
            )
            .where(books_table.c.Titel.ilike(f'%{title}%'))
        )
        results = connection.execute(stmt).fetchall()
        column_names = [
            'ISBN13', 'Titel', 'Språk', 'Pris', 'Utgivningsdatum', 
            'Författare', 'LagerAntal'
        ]
        result_dicts = [
            {column: getattr(book, column) for column in column_names}
            for book in results
        ]
        return result_dicts

def main():
    print("Welcome to the Book Search Program!")
    while True:
        print("\nOptions:")
        print("1. Search for a book by title")
        print("2. Exit")
        choice = input("Enter your choice (1/2): ")

        if choice == '1':
            title = input("Enter the book title to search: ")
            results = search_books_by_title(title)
            if results:
                df = pd.DataFrame(results)
                print("\nSearch Results:")
                print(df)
            else:
                print("\nNo books found with that title.")
            break
        elif choice == '2':
            print("Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()

Welcome to the Book Search Program!

Options:
1. Search for a book by title
2. Exit

Search Results:
               ISBN13                                              Titel  \
0   978-0-13235-088-4                                         Clean Code   
1   978-0-13790-910-0                                               Code   
2   978-1-49204-552-6  Deep Learning for Coders with Fastai and PyTorch:   
3   978-1-80461-387-0                                  Clean Code in PHP   
4   978-0-13235-088-4                                         Clean Code   
5   978-0-13790-910-0                                               Code   
6   978-1-49204-552-6  Deep Learning for Coders with Fastai and PyTorch:   
7   978-1-80461-387-0                                  Clean Code in PHP   
8   978-0-13235-088-4                                         Clean Code   
9   978-0-13790-910-0                                               Code   
10  978-1-49204-552-6  Deep Learning for Coders with Fastai and